In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Tiling system

`pytileproj`'s `tiling_system` module defines several classes representing a multi-level (e.g., zoom, tiling level, ...) projected tiling scheme. A tiling system is a collection of tilings defined in `pytileproj.tiling`, i.e., regular and irregular tilings. These tilings can then be ordered by their tiling/zoom level and forced to only allow certain pixel spacings at a specific level. In the following section, we will look at the most common use case of a tiling system, i.e., a regular projected tiling system. 

## Regular projected tiling system

`pytileproj`'s regular tiling system class `RegularProjTilingSystem` exptects the following arguments:

- `name`: Name of the tiling system.
- `crs`: A spatial reference system represented by anything pyproj supports, e.g., EPSG code, PROJ4 string, WKT string, ...
- `tilings`: Dictionary linking tiling/zoom levels to regular or irregular tilings.
- `proi_zone_geog` (optional): Projection zone in geographic coordinates. If not given, the zone is fetched from EPSG registry.
- `allowed_samplings` (optional): Dictionary linking tiling/zoom levels to a list of allowed samplings/pixel spacings.

### Initialisation

Lets dive deeper into how a `RegularProjTilingSystem` can be used. As an example, we want to create tiling system for the Equi7Grid EU projection. First, we need to define the tilings we want to use within the system:

In [ ]:
from pytileproj.tiling import RegularTiling

extent = [0, 0, 8_700_000, 6_000_000]
axis_orientation = ["E", "N"]
coarse_tiling = RegularTiling(
    name="my_coarse_tiling",
    extent=extent,
    sampling=1_000,
    tile_shape=(300_000, 300_000),
    tiling_level=1,
    axis_orientation=axis_orientation,
)
fine_tiling = RegularTiling(
    name="my_fine_tiling",
    extent=extent,
    sampling=10,
    tile_shape=(100_000, 100_000),
    tiling_level=2,
    axis_orientation=axis_orientation,
)

Then, we can create a `RegularProjTilingSystem` by attaching the respective EPSG code:

In [ ]:
from pytileproj.tiling_system import RegularProjTilingSystem

name = "e7eu"
epsg = 27704
rpts = RegularProjTilingSystem(
    name=name,
    tilings={tiling.tiling_level: tiling for tiling in [coarse_tiling, fine_tiling]},
    crs=epsg,
)
rpts

`RegularProjTilingSystem` also provides another method to create an object from tiling definitions, which directly assume that a regular projected tiling system has the same extent and axis orientation. In addition, the user has control over the tile size in projection units rather than the tile shape in pixels:

In [ ]:
from pytileproj.tiling_system import ProjSystemDefinition, RegularTilingDefinition

rpts_def = ProjSystemDefinition(
    name=name, crs=epsg, extent=extent, axis_orientation=axis_orientation
)
tiling_defs = {
    1: RegularTilingDefinition(name="my_coarse_tiling", tile_shape=300_000),
    2: RegularTilingDefinition(name="my_fine_tiling", tile_shape=100_000),
}
rpts = RegularProjTilingSystem.from_sampling(
    {1: 1_000, 2: 10}, proj_def=rpts_def, tiling_defs=tiling_defs
)

### Projection system interactions

Often it is required to transform coordinates back and forth between different projection systems. `RegularProjTilingSystem` provides several methods to do this in a straighforward way. If you want transform geographic coordinates to projected coordinates, you can use `lonlat_to_xy`:

In [ ]:
lon, lat = 16.3926, 48.1674
proj_coord = rpts.lonlat_to_xy(lon, lat)
proj_coord

The same can be done the other way around:

In [ ]:
rpts.xy_to_lonlat(proj_coord.x, proj_coord.y)

Note that for coordinates outside the projection zone, an error will be raised:

In [ ]:
lon, lat = -100, 30
rpts.lonlat_to_xy(lon, lat)

You can check in advance if a point or geometry is within the projection by using Python's `in` operator:

In [ ]:
from pytileproj import ProjCoord

coord = ProjCoord(lon, lat, 4326)
coord in rpts

If you want to know the units of the projection system, you can retrieve this information via:

In [ ]:
rpts.unit

### Tiles

Regular projected tiling systems create tiles on the fly, since their regular tiling scheme allows an efficient computation of tile properties. A tile can be created by providing a location of interest with the following methods: `get_tile_from_lonlat`, `get_tile_from_xy`, and `get_tile_from_coord`. Here is an example:

In [ ]:
lon, lat = 16.3926, 48.1674
rpts.get_tile_from_lonlat(lon, lat, tiling_id=1)

In addition, you can create a tile by using an OGC tile index (x, y, z):

In [ ]:
rpts.get_tile_from_index(17, 5, 1)

If you are interested in retrieving a list of tiles intersecting with a region of interest, you can use `get_tiles_in_geog_bbox`. It returns a generator to enable lazy tile retrieval.

In [ ]:
list(rpts.get_tiles_in_geog_bbox([16, 48, 18, 50], tiling_id="my_fine_tiling"))

If you want to know if the tilings withing the tiling systems are congruent, i.e., tiles at a higher tiling level (fine tiling) are multiples of tiles at a lower tiling level (coarse tiling).

In [ ]:
rpts.is_congruent

### Export

A tiling system offers several export methods, e.g., an export to a `GeoDataFrame` (`to_geodataframe`) or a shapefile (`to_shapefile`). Here is an example how to generate a `GeoDataFrame`:

In [ ]:
rpts.to_geodataframe()

If you want to share an OGC compliant definition of the tiling system, you can use `to_ogc_standard` or `to_ogc_json`:

In [ ]:
import json
import pprint
from pathlib import Path

ogc_ts_path = Path("my_tiling_system.json")
rpts.to_ogc_json(ogc_ts_path)

with ogc_ts_path.open() as f:
    ogc_ts = json.load(f)

pprint.pprint(ogc_ts)  # noqa: T203

In [ ]:
ogc_ts_path.unlink(missing_ok=True)

### Visualisation

A `RegularProjTilingSystem` can be also visualised on a map in a similar manner as a `RasterTile` object, if the optional dependencies `matplotlib` and `cartopy` are installed.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 28))
rpts.plot(
    tiling_id=1,
    label_tile=True,
    label_size=5,
    plot_zone=True,
    facecolor="#2cee768f",
    alpha=0.6,
    extent=extent,
)